### Downloading and installing the Tortoise-TTS model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/tortoise-tts
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.0.post2-py3-none-any.whl (253 kB)
  Using cached librosa-0.10.0.post1-py3-none-any.whl (252 kB)
  Using cached librosa-0.10.0-py3-none-any.whl (252 kB)
  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
INFO: pip is looking at multiple versions of resampy to determine which version is compatible with other requirements. This could take a while.
  Using cached resampy-0.4.1-py3-none-any.whl (3.1 MB)
  Using cached resampy-0.4.0-py3-none-any.wh

In [ ]:
import IPython

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

### Upload your audios samples here

In [ ]:
!gdown 1Qqi4O6BfSAjx_t1bQ-OmiSo4ZRNa6Qq9 # task_record.wav
!gdown 17Q9c54-kIkdiK52K_DWXkCtYTyv4ufM2 # task_record_Voice.wav
!gdown 1Ss-vADzRayNq3v0pmIubT7qgmbXap2ha # translated_whisper_medium.pkl

Downloading...
From: https://drive.google.com/uc?id=1Qqi4O6BfSAjx_t1bQ-OmiSo4ZRNa6Qq9
To: /content/tortoise-tts/task_record.wav
100% 121M/121M [00:00<00:00, 208MB/s]
Downloading...
From: https://drive.google.com/uc?id=17Q9c54-kIkdiK52K_DWXkCtYTyv4ufM2
To: /content/tortoise-tts/task_record_Vocals.wav
100% 111M/111M [00:00<00:00, 193MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ss-vADzRayNq3v0pmIubT7qgmbXap2ha
To: /content/tortoise-tts/translated_whisper_medium.pkl
100% 9.52k/9.52k [00:00<00:00, 28.1MB/s]


In [ ]:
!gdown 1-0S4DbWxvbd_fjKld--SrTzVNKgaE6DD # whole text

Downloading...
From: https://drive.google.com/uc?id=1-0S4DbWxvbd_fjKld--SrTzVNKgaE6DD
To: /content/tortoise-tts/translated_whole_speech_medium.pkl
100% 9.25k/9.25k [00:00<00:00, 26.4MB/s]


In [ ]:
path2data = "/content/tortoise-tts/task_record_Vocals.wav"

signal, sample_rate = torchaudio.load(path2data)
default_sample_rate = 22050
resampler = torchaudio.transforms.Resample(sample_rate, default_sample_rate)
signal = torch.mean(signal, dim=0, keepdim=True)
signal = resampler(signal)
signal = signal.numpy().squeeze()

In [ ]:
from scipy.io import wavfile
import numpy as np
import os

CUSTOM_VOICE_NAME = "shulman"
custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
if not os.path.exists(custom_voice_folder):
  os.makedirs(custom_voice_folder)
for i in np.arange(20,70,10):
  file_name = f"{custom_voice_folder}/tortoise_22050_{i}_{i+10}.wav"
  wavfile.write(file_name, default_sample_rate, signal[default_sample_rate*i:default_sample_rate*(i+11)])

In [ ]:
import pickle
path2sentences = '/content/drive/MyDrive/outputs/translation_result.pkl'
with open(f'{path2sentences}', 'rb') as file:
    whole_text = pickle.load(file)

In [ ]:
whole_text

'Today we will try to talk about how we can project our knowledge of the present for the future and whether we can.The topic is formulated as political forecasting.I, as a political scientist, naturally bow to my professional side all the time.But the basic principles that we are trying to set out today are, in general, universally applicable to forecasting, that is, to trying to see the future in any sphere of human life.Let\'s remember a little bit of what we agreed on in our class yesterday, when we talked about these cognitive distortions, the complexities, the perception of us, the information that concerns us.The whole volume, the whole wide set of cognitive distortions that we listed.Yesterday, in general, what the brain wants to think about is the conventional patters, perceives what\'s normal as safe, picks up information, and filters information that supports the beliefs, the perceptions that we have already developed, and that the same experience, the limited perspective, th

### Generate longer speech from a text file

In [ ]:
from tortoise.utils.text import split_and_recombine_text
from time import time
import os

outpath = "/content/drive/MyDrive/outputs/tortoise_tts_result/results/longform/"
if not os.path.exists(outpath):
  os.makedirs(outpath)

In [ ]:
texts = split_and_recombine_text(whole_text)

In [ ]:
len(texts)

35

In [ ]:
seed = int(time())

voice_outpath = os.path.join(outpath, CUSTOM_VOICE_NAME)
os.makedirs(voice_outpath, exist_ok=True)

voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

In [ ]:
for j in range(1, len(texts)):
    text = texts[j]
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                              preset="standard", k=1, use_deterministic_seed=seed)
    gen = gen.squeeze(0).cpu()
    torchaudio.save(os.path.join(voice_outpath, f'{j}.wav'), gen, 24000)

Generating autoregressive samples..


100%|██████████| 16/16 [04:39<00:00, 17.50s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 16/16 [00:33<00:00,  2.10s/it]


Transforming autoregressive outputs into audio..


...

 69%|██████▉   | 11/16 [00:22<00:10,  2.09s/it]

No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.


100%|██████████| 16/16 [00:33<00:00,  2.08s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 200/200 [03:32<00:00,  1.06s/it]


In [ ]:
import IPython


In [ ]:
IPython.display.Audio('/content/drive/MyDrive/outputs/tortoise_tts_result/results/longform/shulman/1.wav')

In [ ]:
whole_text

'Today we will try to talk about how we can project our knowledge of the present for the future and whether we can.The topic is formulated as political forecasting.I, as a political scientist, naturally bow to my professional side all the time.But the basic principles that we are trying to set out today are, in general, universally applicable to forecasting, that is, to trying to see the future in any sphere of human life.Let\'s remember a little bit of what we agreed on in our class yesterday, when we talked about these cognitive distortions, the complexities, the perception of us, the information that concerns us.The whole volume, the whole wide set of cognitive distortions that we listed.Yesterday, in general, what the brain wants to think about is the conventional patters, perceives what\'s normal as safe, picks up information, and filters information that supports the beliefs, the perceptions that we have already developed, and that the same experience, the limited perspective, th